In [ ]:
# Instalación de librerías requeridas
%pip install sqlalchemy
%pip install psycopg2
%pip install psycopg2-binary
%pip install scikit-learn
%pip install pandas

In [2]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


In [ ]:
with open('../../configBD/config.yml', 'r') as f:
    config      = yaml.safe_load(f)
    config_etl  = config['bodega']
    config_bd   = config['mensajeria']
config_etl      # solo para verificar


In [ ]:
url_bd = (
    f"{config_bd['driver']}://{config_bd['user']}:{config_bd['password']}@"
    f"{config_bd['host']}:{config_bd['port']}/{config_bd['db']}"
)
url_bd


In [ ]:
url_etl = (
    f"{config_etl['driver']}://{config_etl['user']}:{config_etl['password']}@"
    f"{config_etl['host']}:{config_etl['port']}/{config_etl['db']}"
)
url_etl


In [6]:
cliente_bd  = create_engine(url_bd)   # fuente operativa
cliente_etl = create_engine(url_etl)  # data-warehouse / ETL


In [ ]:
dim_tiempo = pd.read_sql_table('dim_tiempo', url_etl)

dim_tiempo = dim_tiempo.rename(columns={"Año":"year", "Mes":"month", "Dia":"day"})
dim_tiempo['date'] = pd.to_datetime(dim_tiempo[['year','month','day']]).dt.date
dim_tiempo.head()


In [8]:
novedades = pd.read_sql_table('mensajeria_novedadesservicio', url_bd)


In [9]:
novedades['fecha_novedad'] = pd.to_datetime(novedades['fecha_novedad']).dt.date


In [ ]:
print("Rango de fechas en dim_tiempo:", dim_tiempo['date'].min(), "→", dim_tiempo['date'].max())
print("Rango de fechas en novedades:", novedades['fecha_novedad'].min(), "→", novedades['fecha_novedad'].max())


In [11]:
# lista corta de stop-words en español
stop_words_spanish = [
    'el','la','los','las','de','del','un','una','por','para','con','sin',
    'y','o','en','que','no','se','a','al','me','lo','este','esto','estas'
]

# 1) limpiar texto
novedades['descripcion_limpia'] = novedades['descripcion'].fillna('').str.lower()

# 2) vectorizar TF-IDF
tfidf = TfidfVectorizer(stop_words=stop_words_spanish, max_features=1000)
X     = tfidf.fit_transform(novedades['descripcion_limpia'])

# 3) K-means
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
novedades['cluster'] = kmeans.fit_predict(X)

# 4) mapear cluster → categoría
cluster_names = {0:'problemas_tecnicos', 1:'reprogramaciones', 2:'esperas',
                 3:'entregas', 4:'otros'}
novedades['categoria'] = novedades['cluster'].map(cluster_names)


In [ ]:
# 1) agrupar por fecha_novedad + categoría
novedades_agrupadas = (
    novedades.groupby(['fecha_novedad','categoria'])
             .size()
             .unstack(fill_value=0)
             .reset_index()
)

# 2) añadir clave de tiempo
novedades_agrupadas = novedades_agrupadas.merge(
    dim_tiempo[['tiempo_key','date']],
    left_on='fecha_novedad', right_on='date', how='left'
).rename(columns={'tiempo_key':'TiempoKey'}).drop(columns=['date'])

# 3) guarda en Data Warehouse
novedades_agrupadas.to_sql('fact_novedades_clustering',
                           cliente_etl,
                           if_exists='replace',
                           index=False)

# 4) opcional: exportar CSV
# novedades_agrupadas.to_csv('novedades_agrupadas_clustering.csv', index=False)

novedades_agrupadas.head()
